# Bike Share Analysis in Python
I learned a lot working on the bike share case study for my Google Data Analytics certificate. I got to apply the skills I learned from the course and experienced first hand how data can impact decisions. Working on the project allowed me to develop a strong foundation with R including how to wrangle, clean, and visualize data. 
I know that Python is another programming language used to analyze data and I want to redo the same project to demonstrate my ability with Python as well as to have a better understanding of the differences between R and Python. 

# Background
A little background on the scenario and data:

Cyclistic is a bike-share company in Chicago with a fleet of 5,824 geotracked bicycles and a network of 692 stations. They offer 3 pricing plans: single-ride passes, full-day passes, and annual memberships. Customers who purchase single-ride or full-day passes are referred to as casual riders and customers who purchase annual memberships are referred to as Cyclistic members. The company determined that annual members are more profitable than casual riders and is aiming to create a marketing campaign to maximize the number of annual members by converting casual riders.

Key stakeholder: Lily Moreno, the director of marketing responsible for developing campaigns to promote bike-share program.

Business task: How do annual members and casual riders use Cyclistic bikes differently?

***
I used proxy data from a similar bike sharing company called Divvy which can be downloaded here: <a href="https://divvy-tripdata.s3.amazonaws.com/index.html" target="_blank">https://divvy-tripdata.s3.amazonaws.com/index.html</a>
I downloaded 12 months of data and merged 12 CSV files into a dataframe.

In [1]:
# Setup envionment and import libraries
from matplotlib import pyplot as plt 
import pandas as pd
import seaborn as sns
import numpy as np
import glob
from pandas.api.types import CategoricalDtype

# Full output for each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Read and merge CSV files to dataframe
df = pd.concat(map(pd.read_csv, glob.glob("data/*.csv")))

I wanted to view the data to confirm the merge was successful and to review the column names and data types. 

In [3]:
# Preview data
df
df.info()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,47EC0A7F82E65D52,classic_bike,2022-03-21 13:45:01,2022-03-21 13:51:18,Wabash Ave & Wacker Pl,TA1307000131,Kingsbury St & Kinzie St,KA1503000043,41.886875,-87.626030,41.889177,-87.638506,member
1,8494861979B0F477,electric_bike,2022-03-16 09:37:16,2022-03-16 09:43:34,Michigan Ave & Oak St,13042,Orleans St & Chestnut St (NEXT Apts),620,41.900998,-87.623752,41.898203,-87.637536,member
2,EFE527AF80B66109,classic_bike,2022-03-23 19:52:02,2022-03-23 19:54:48,Broadway & Berwyn Ave,13109,Broadway & Ridge Ave,15578,41.978353,-87.659753,41.984045,-87.660274,member
3,9F446FD9DEE3F389,classic_bike,2022-03-01 19:12:26,2022-03-01 19:22:14,Wabash Ave & Wacker Pl,TA1307000131,Franklin St & Jackson Blvd,TA1305000025,41.886875,-87.626030,41.877708,-87.635321,member
4,431128AD9AFFEDC0,classic_bike,2022-03-21 18:37:01,2022-03-21 19:19:11,DuSable Lake Shore Dr & North Blvd,LF-005,Loomis St & Jackson Blvd,13206,41.911722,-87.626804,41.877945,-87.662007,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
634853,8891BA0053ECEC4F,electric_bike,2022-05-27 22:00:02,2022-05-27 22:07:01,Clark St & Newport St,632,NaN,NaN,41.944557,-87.654830,41.920000,-87.650000,member
634854,47D8B5FBCADECFC1,electric_bike,2022-05-15 16:05:39,2022-05-15 16:44:12,Clark St & Newport St,632,NaN,NaN,41.944479,-87.654758,41.920000,-87.760000,member
634855,AA8D16CF38B40703,electric_bike,2022-05-21 10:10:13,2022-05-21 10:26:09,Francisco Ave & Bloomingdale Ave,429,NaN,NaN,41.910000,-87.700000,41.920000,-87.660000,casual
634856,897EBFD44F329E0A,electric_bike,2022-05-12 07:53:58,2022-05-12 08:01:18,Francisco Ave & Bloomingdale Ave,429,NaN,NaN,41.910000,-87.700000,41.900000,-87.690000,member


In [6]:
# Convert started_at and ended_at columns to DateTime format
df['started_at'] = pd.to_datetime(df['started_at'], format = '%Y-%m-%d %H:%M:%S')
df['ended_at'] = pd.to_datetime(df['ended_at'], format = '%Y-%m-%d %H:%M:%S')

In [7]:
df1 = df
# Add columns for year, month, day, and day of the week. 
df1['year'] = df1['started_at'].dt.year
df1['month'] = df1['started_at'].dt.month
df1['day'] = df1['started_at'].dt.day
df1['day_of_week'] = df1['started_at'].dt.day_name()

# Order days of the week
cats = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
cat_type = CategoricalDtype(categories = cats, ordered = True)
df1['day_of_week'] = df1['day_of_week'].astype(cat_type)


In [8]:
# Add column for ride length
df1['ride_length'] = df1['ended_at'] - df1['started_at']
# Convert DateTime format to seconds (int)
df1['ride_length'] = df1['ride_length'].dt.total_seconds()

In [ ]:
# Remove irrelevant columns and missing rows
df1.drop(['start_lat', 'start_lng', 'end_lat', 'end_lng'], axis = 1, inplace = True)
df1.dropna(subset = ['start_station_name', 'end_station_name'])


In [ ]:
# Check for duplicate rows
df_dupes = df1[df1.duplicated(['ride_id'])]
print(df_dupes)

In [ ]:
#Check for inconsistent data (i.e. more than 2 member types)
df1['member_casual'].value_counts()
df1['rideable_type'].value_counts()

In [ ]:
# Check data is within date range
df1['started_at'].max()
df1['started_at'].min()

In [ ]:
# Check for negative ride durations
negative_ride_length = df1[(df1['ride_length'].dt.total_seconds() < 0)]
print(negative_ride_length)

In [ ]:
# Drop rows with docked_bike or negative ride length
df1 = df1[(df1['rideable_type'] != 'docked_bike') & (df1['ride_length'].dt.total_seconds() > 0)]

In [ ]:
df1['ride_length'].describe()

In [ ]:
# Group by day of week and member type, then aggregate average ride length
df2 = df1.groupby(['day_of_week', 'member_casual'])['ride_length'].mean()
print(df2)

In [ ]:
#Plot average ride length vs day of the week
plt.figure(figsize=[15, 14])
sns.barplot(data = df1, x = 'day_of_week', y = 'ride_length', hue = 'member_casual')
plt.title('Average Ride Length vs. Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Ride Length (seconds)')

In [ ]:
#Plot number of rides vs. day of the week
plt.figure(figsize=[15, 14])
sns.countplot(data = df1, x = 'day_of_week', hue = 'member_casual')